In [2]:
import os

In [3]:
%pwd

'c:\\Users\\vvkar\\OneDrive\\Desktop\\major project\\Abstractive-Text-Summarizer\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\vvkar\\OneDrive\\Desktop\\major project\\Abstractive-Text-Summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [7]:
from Abstractive_Text_Summarizer.constants import *
from Abstractive_Text_Summarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

[2025-01-09 11:19:20,285: INFO: config: PyTorch version 2.4.1 available.]


In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            eval_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()
        


        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-12-01 12:30:52,740: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-01 12:30:52,752: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-01 12:30:52,753: INFO: common: created directory at: artifacts]
[2024-12-01 12:30:52,762: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\vvkar\miniconda3\envs\AbsTextSumm\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\vvkar\AppData\Local\Temp\ipykernel_14440\177660330.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


  0%|          | 0/920 [00:00<?, ?it/s]

{'loss': 3.1995, 'grad_norm': 322.5968322753906, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 3.2262, 'grad_norm': 154.7872314453125, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}
{'loss': 2.9225, 'grad_norm': 180.6544189453125, 'learning_rate': 3e-06, 'epoch': 0.03}
{'loss': 2.8533, 'grad_norm': 244.4595184326172, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.04}
{'loss': 2.6912, 'grad_norm': 227.75010681152344, 'learning_rate': 5e-06, 'epoch': 0.05}
{'loss': 2.7275, 'grad_norm': 168.98187255859375, 'learning_rate': 6e-06, 'epoch': 0.07}
{'loss': 2.6254, 'grad_norm': 200.16253662109375, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.08}
{'loss': 2.4282, 'grad_norm': 175.9784393310547, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.09}
{'loss': 2.4937, 'grad_norm': 125.54471588134766, 'learning_rate': 9e-06, 'epoch': 0.1}
{'loss': 2.4588, 'grad_norm': 112.201416015625, 'learning_rate': 1e-05, 'epoch': 0.11}
{'loss': 2.2212, 'grad_norm': 118.20